# CORS

> Python tool library to download collections of NOAA NGS RINEX datafiles from the CORS site.

In [ ]:
#| default_exp CORS

In [ ]:
#| hide
from nbdev.showdoc  import *
from fastcore.test  import *
from fastcore.utils import *

In [ ]:
#| hide
import nbdev

In [ ]:
#| hide
nbdev.nbdev_export()

# Code

#| hide
## imports

#| hide
### imports, public

In [ ]:
#| export
import datetime
import time
import os
import glob
import inspect
import ipywidgets       as ipw
import multiprocessing
import numpy            as np
import pandas           as pd
import panel            as pn
import re
import requests
import subprocess       as sp

#| hide
### from public

In [ ]:
#| export
from   bs4              import BeautifulSoup
from   pathlib          import Path

#| hide
### imports, conditional

In [ ]:
#| export
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

## Module Globals

`CORS_Lib_asof` is the date CORS library was compiled.

`IN_COLAB` will be `True` if you are running in Google COlab
and set to `False` otherwise.

In [ ]:
#| exports
CORS_Lib_asof      = 'CORS_Lib_asof: 2023-0614-1908'

The following module global variables hold the 
urls to the various parts of the NOAA CORS website.

In [ ]:
#| exports
base_url           = 'https://geodesy.noaa.gov/'
rinex_url          = base_url+'corsdata/rinex/'
coords_url         = base_url+'corsdata/coord/coord_14/'
plots_url          = base_url+'corsdata/Plots/'
station_log_url    = base_url+'corsdata/station_log/'
cors_info_url_list = [ base_url+'/corsdata/readme.txt', 
                      base_url+'/corsdata/RINEX211.txt' ]

`test_folder` is the default folder where downloaded CORS data will be stored
during testing of this package.

In [ ]:
#| exports
test_folder = '/tmp/test/'

`wget_return_codes` are the code numbers and decoded descriptions returned by
the [wget](https://www.gnu.org/software/wget/) program.  wget is used by this
package to download the datasets.

In [ ]:
#| exports
wget_return_codes = ('0, No problems occurred.', 
                     '1, Generic error code.',
                     '2, Parse error---for instance, when parsing command-line options, the .wgetrc or .netrc...',
                     '3, File I/O error.',
                     '4, Network failure.',
                     '5, SSL verification failure.',
                     '6, Username/password authentication failure.',
                     '7, Protocol errors.',
                     '8, Server issued an error response.')

[`Teqc`](https://www.unavco.org/software/data-processing/teqc/teqc.html) 
is used to slice RINEX files by time.  The url list below are the full list
of UNAVCO download sites for `teqc`.  The `install_teqc` function will download 
try each one and check that it works.

In [ ]:
#| exports
teqc_list          = [ 
    'https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64s.zip',
    'https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64d.zip',
    'https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64d.zip',
    'https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64s.zip'
]

If running on Google Colab, the default path to store binary programs such
as teqc is set below.  If not running in Colab, binarys will be stored in 
`~/bin` in the users home directory.

In [ ]:
#| export
if IN_COLAB:
  local_bin = '/usr/local/bin'
else:
  local_bin = f'{str(Path.home())}/bin/'

In [ ]:
print(f'     IN_COLAB: {IN_COLAB}')
print(f'    local_bin: {local_bin}')
print(f'CORS_Lib_asof: {CORS_Lib_asof}')
print(f'     base_url: {base_url}')
print(f'    rinex_url: {rinex_url}')

     IN_COLAB: False
    local_bin: /home/wright/bin/
CORS_Lib_asof: CORS_Lib_asof: 2023-0614-1908
     base_url: https://geodesy.noaa.gov/
    rinex_url: https://geodesy.noaa.gov/corsdata/rinex/


## Functions

#| hide
### def install_teqc():
See: https://www.unavco.org/software/data-processing/teqc/teqc.html

In [ ]:
#| export
# Get and install teqc
def install_teqc() ->str:
  '''
  This function is used to download and install the teqc software from 
  Unavco. It first checks if the teqc software is already installed
  in the system. If not, it downloads the teqc_Lx86_64s.zip file from
  Unavco, unzips it, creates a local bin directory and moves the teqc
  file to the local bin directory. Finally, it deletes the zip file.

  See: https://www.unavco.org/software/data-processing/teqc/teqc.html
  

  Parameters:
  None

  Returns:
  A string indicating the state of the installation. 
  It can be either: 'Teqc installed.', 'Teqc was already installed.', or
  'teqc install failed.'
  '''
  state = 'initial.'
  os.chdir('/tmp/')
  print(f'install_teqc(): ')
  rv = os.path.isfile(f'{local_bin}teqc')
  if rv == False:
    state = 'teqc install failed.'
    for teqc_zip in teqc_list:
        print(f'Downloading {teqc_zip} and installing Teqc from Unavco.  ')
        sp.run(f'wget {teqc_zip}',                  shell=True)
        teqc_zip_fn = teqc_zip.split( '/')[-1]
        rv = sp.run(f'unzip -o {teqc_zip_fn}',       shell=True)
        sp.run(f'rm -rf {teqc_zip_fn}',              shell=True)
        rv = sp.run(f'/tmp/teqc -help',              shell=True, capture_output=True )
        if rv.returncode == 0 :
            print('Installing teqc.')
            sp.run(f'mkdir -p {local_bin}',          shell=True)
            sp.run(f'mv -f teqc {local_bin}',        shell=True)  
            state = ' Teqc installed.'
            break
        else:
            print(f'Install failed.  rv.return={rv.returncode}')
            print(rv)
  else:
    a = 1;
    state='Teqc was already installed.'
  return state

In [ ]:
test_teqc_state = install_teqc()
print(test_teqc_state)
! which teqc
! teqc -version

install_teqc(): 
Teqc was already installed.
/home/wright/bin/teqc
executable:  teqc
version:     teqc  2019Feb25
build:       Linux 2.6.32-573.12.1.x86_64|x86_64|gcc|Linux 64|=+


#| hide
### def extract_year( date ):

In [ ]:
#| export
def extract_year( date:str ):
  """
  Extract the year from a date string in the form: `"2023/1/1"`
  """
  Year = date.split('/')[0]
  return Year

In [ ]:
#| hide 
test_eq(extract_year( '2023/1/20'), '2023')

Here is an example.

In [ ]:
extract_year( '2023/1/20')

'2023'

#| hide
### def isTimeFormat(ts):

In [ ]:
#| export
def isTimeFormat(
  ts:str   # Time string as: HH:MM:SS
) ->bool:  # True if valid HH:MM:SS time string.
  """
  isTimeFormat(ts) checks the 'ts' input string for validity. A valid string is in
  this format: '13:34:56'.

  Returns True for a valid string,a nd False for invalid.
  """
  t = []
  rv = False;
  try:
      t = time.strptime(ts, '%H:%M:%S')
      rv = True;
  except ValueError:
      rv = False
  return rv

Below we test the `isTimeFormat()` with several input times. 

In [ ]:
print('          Returned\n'\
      'Time      Value (rv)\n'\
      '--------------------')
for t in ['12:34:56',
          '12:34',
          '01:02:03',
          '23:01:02',
          '25:34:63']:
  rv = isTimeFormat(t)
  print(f'{t:9} rv:{rv}')

          Returned
Time      Value (rv)
--------------------
12:34:56  rv:True
12:34     rv:False
01:02:03  rv:True
23:01:02  rv:True
25:34:63  rv:False


In [ ]:
#| hide 
# Test isTimeFormat()
test_eq( isTimeFormat('12:34:56'), True)
test_eq( isTimeFormat('12:34'),   False)
test_eq( isTimeFormat('01:02:03'), True)
test_eq( isTimeFormat('01:02:03'), True)

for t, trv, c in [
              ('12:34:56', True,  ""),
              ('12:34',   False,  "Has no seconds."),
              ('01:02:03', True,  ""),
              ('23:01:02', True,  ""), 
              ('25:34:63', False, "Seconds greater than 59")
             ]:
  rv = isTimeFormat(t)
  test_eq(rv, trv)
  print(f'{t:9} rv:{rv} {trv}, {c}')

12:34:56  rv:True True, 
12:34     rv:False False, Has no seconds.
01:02:03  rv:True True, 
23:01:02  rv:True True, 
25:34:63  rv:False False, Seconds greater than 59


#| hide
### def utctoweekseconds(utc,leapseconds):

In [ ]:
#| export
def utctoweekseconds(
  utc,                  # Date string 'YYYY/MM/DD'
  leapseconds           # For 2023, use -18
  ) ->tuple:            # (GPS_Week, GPS_day, Seconds_since_midnight)
  '''
    This function takes two parameters, utc and leapseconds, and returns the
    GPS week, the GPS day, and the seconds and microseconds since the beginning of the GPS week.

    Parameters:
    utc (datetime.datetime): The UTC time to be converted.
    leapseconds (int): The number of leap seconds to be added to the UTC time.
    See: 
    [GPS Leap Seconds](https://shorturl.at/jozO3)
    for more information.

    Returns:
    A tuple containing the GPS week, the GPS day, and the seconds and 
    microseconds since the beginning of the GPS week.

    The GPS week is an integer representing the number of weeks since the 
    beginning of the GPS epoch (1980-01-06 00:00:00).
    The GPS day is an integer representing the number of days since the 
    beginning of the GPS week.
    The seconds and microseconds since the beginning of the GPS week are both
    integers representing the number of seconds and microseconds since the
    beginning of the GPS week.

    Example:
    utc = datetime.datetime(2020, 1, 1, 0, 0, 0)
    leapseconds = 18

    utctoweekseconds(utc, leapseconds)

    Returns: (2086, 0, 0, 0)
  '''
  import datetime, calendar
  datetimeformat = "%Y-%m-%d %H:%M:%S"
  epoch = datetime.datetime.strptime("1980-01-06 00:00:00",datetimeformat)
  tdiff = utc -epoch  + datetime.timedelta(seconds=leapseconds)
  gpsweek = tdiff.days // 7 
  gpsdays = tdiff.days - 7*gpsweek         
  gpsseconds = tdiff.seconds + 86400* (tdiff.days -7*gpsweek) 
  return gpsweek,gpsdays,gpsseconds,tdiff.microseconds


#| hide
### def compute_day_of_year(date_str):

In [ ]:
#| export
def compute_day_of_year(
  date_str:str  # Date as 'YYYY/MM/DD'
) ->tuple:
  '''
    This function computes the day of the year from a given date
    string in the format 'YYYY/MM/DD'.

    Parameters:
    date_str (str): A string representing a date in the format 'YYYY/MM/DD'.

    Returns:
    A tuple containing the date in the following formats:
    ( YYYY/jjj,   YYYY-MMDD-Jjjj,   YYYY,   jjj,        DoW)', 
    where `YYYY` is the year, `jjj`: Juliean Day, `DoY` is the Day of the week. 
    
    Each tuple element has a particular purpose. The first element `YYYY/jjj` 
    represents the portion of the path used on the CORS web site.  The second
    element, `YYYY-MMDD-Jjjj` is used to generate the path to store downloaded
    data.  The `YYYY` is simply the year, and `jjj` is simple the Julean day of 
    the year, and `DoW` is the day of the week.  Each element is type str.


    Example:
    compute_day_of_year('2020/04/15')

    Returns:
    ('2020/105', '2020-0415-J105', '2020', '105', '04', '15')

    The returned tuple contains the day of the year in the format 'YYYY/DDD', 
    the year as a string, the day as a string, the month as a string, 
    and the day as a string. The first element of the tuple is the day 
    of the year in the format 'YYYY/DDD', which is the year followed by the 
    day of the year. The second element of the tuple is the year as a string. 
    The third element of the tuple is the day as a string. The fourth element
    of the tuple is the month as a string. The fifth element of the tuple is
    the day of the week as a string.
    
    The CORS path segment refers to the portion of the url that represents the
    date of the date.

  '''
  JDay = int(( datetime.datetime.strptime(date_str,'%Y/%m/%d') - datetime.datetime(int(date_str.split('/')[0]),1,1)).days + 1 )
  JDay = f'{JDay:03d}'
  date_parts =  date_str.split('/')
  Year = date_parts[0]
  Month = f'{int(date_parts[1]):02d}'
  Day   = f'{int(date_parts[2]):02d}'
  return f'{Year}/{JDay}', f'{Year}-{Month}{Day}-J{JDay}', Year, JDay, Month, Day

In [ ]:
test_dates = [
    '2023/1/1',
    '2023/01/01',
    '2023/12/31',
    '2023/05/20'
]
print('All returned data:')
print('Input Date     ( YYYY/jjj,   YYYY-MMDD-Jjjj,   YYYY,   jjj,        DoW)')
print('-----------------------------------------------------------------------')
for d in test_dates:
  print( f'{d:12} : { compute_day_of_year(d) } ')

print('\nDate           CORS path\n'\
       '               segment only:')
for d in test_dates:
  print( f'{d:12} : { compute_day_of_year(d)[0]}')

All returned data:
Input Date     ( YYYY/jjj,   YYYY-MMDD-Jjjj,   YYYY,   jjj,        DoW)
-----------------------------------------------------------------------
2023/1/1     : ('2023/001', '2023-0101-J001', '2023', '001', '01', '01') 
2023/01/01   : ('2023/001', '2023-0101-J001', '2023', '001', '01', '01') 
2023/12/31   : ('2023/365', '2023-1231-J365', '2023', '365', '12', '31') 
2023/05/20   : ('2023/140', '2023-0520-J140', '2023', '140', '05', '20') 

Date           CORS path
               segment only:
2023/1/1     : 2023/001
2023/01/01   : 2023/001
2023/12/31   : 2023/365
2023/05/20   : 2023/140


In [ ]:
test_eq(1,1)

#| hide
### def getHTMLdocument(url):

In [ ]:
#| export
def getHTMLdocument(
  url      # The `url` to get. 
) -> str:  # the contents of the url as a string.
    response = requests.get(url)  # response will be provided in JSON format
    return response.text

In [ ]:
# Test  getHTMLdocument(rinex_url)
test_html = getHTMLdocument(rinex_url)

i = 0
for line in test_html.splitlines():
  print(line); i += 1
  if i == 10:
    break

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<html>
 <head>
  <title>Index of /corsdata/rinex</title>
 </head>
 <body>
<h1>Index of /corsdata/rinex</h1>
  <table>
   <tr><th valign="top"><img src="/icons/blank.gif" alt="[ICO]"></th><th><a href="?C=N;O=D">Name</a></th><th><a href="?C=M;O=A">Last modified</a></th><th><a href="?C=S;O=A">Size</a></th><th><a href="?C=D;O=A">Description</a></th></tr>
   <tr><th colspan="5"><hr></th></tr>


#| hide
### def CORS_get_Support_url_list( base_url, date, file_types="sp3.gz|n.gz|g.gz" ):
See: https://www.geeksforgeeks.org/beautifulsoup-scraping-link-from-html/

In [ ]:
#| export
def CORS_get_Support_url_list( 
  base_url:str,                        # COR url.
  date:str,                            # Date. Format: 'YYYY:MM:DD'
  file_types:str="sp3.gz|n.gz|g.gz"    # file types to get.
) -> list:                             # A list of CORS nav and SP3 files.
  """
  """
  data_url = base_url+compute_day_of_year( date )[0]
  hdoc = getHTMLdocument(data_url)
  soup = BeautifulSoup( hdoc, 'html.parser')
  urls = []
  for link in soup.find_all('a', attrs={'href' : re.compile(file_types, re.IGNORECASE) } ):
    #print(link.prettify())
    fn = link.get('href')
    url = f'{data_url}/{fn}'
    urls.append(url)
  return urls

In [ ]:
print('Testing: CORS_get_Support_url_list()')
print('sp3.gz and n.gz and g.gz')

url = rinex_url
sp3_and_nav_urls = CORS_get_Support_url_list(url, date='2023/05/23')
for fn in sp3_and_nav_urls:
  print(fn)

print('\nsp3 only.')
file_types="sp3.gz"
sp3_urls = CORS_get_Support_url_list(url, date='2023/05/23', file_types="sp3.gz")
for fn in sp3_urls:
  print(fn)

print('\nn.gz and g.gz only.')
file_types="sp3.gz"
nav_urls = CORS_get_Support_url_list(url, date='2023/05/23', file_types="n.gz|g.gz")
for fn in nav_urls:
  print(fn)
print('Done.')

Testing: CORS_get_Support_url_list()
sp3.gz and n.gz and g.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/IGS0OPSFIN_20231430000_01D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/IGS0OPSRAP_20231430000_01D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/IGS0OPSULT_20231430000_02D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/IGS0OPSULT_20231430600_02D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/IGS0OPSULT_20231431200_02D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/IGS0OPSULT_20231431800_02D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/brdc1430.23g.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/brdc1430.23n.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/igr22632.sp3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/igs22632.sp3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/igu22632_00.sp3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/143/igu22632_06.sp3

#| hide
### def download_urls(url_list, root_dir):
Notes:
1. It is important for a url to end in `/` 
when using recursive `-r` and `-np` options to `wget`. See: https://stackoverflow.com/questions/19004809/using-wget-to-recursively-fetch-a-directory-with-no-parent for details.

In [ ]:
#| export
def download_urls(
  url_list,            # List of urls to download.
  folder,              # Folder to store the downloads.
  clobber=False,       # True to overwrite, False to not overwrite existing files.
  id='',               # Optional `id` to use in the log file to identify the log.
  options='',          # `wget` options.
  cut_dirs=4           # Number of url subdirectories to trim.
) ->list:              # a list of return status strings for each download.
  '''
download_urls() is a function that downloads a list of URLs to a specified root directory.

Parameters:
url_list (list): A list of URLs to be downloaded.
root_dir (str): The root directory to which the URLs will be downloaded.
clobber (bool): A boolean value indicating whether existing files should be overwritten.
id (str): An optional identifier for the download.
options (str): An optional string of additional wget options.
    
    The function uses the wget command to download the URLs. The wget options used are:
    -nv: Not verbose
    -nc: No clobber (use for OSB files to avoid overwriting 1-Sec with 30-Sec)
    -np: No parent. (Important)
    -nH: Do not create a directory named after the url
    --cut-dirs=4: Remove the first four subdirs
    -R "index.html*": Do not keep the index.html* files.
    -P xxx: xxx will be the root directory

    Wget return codes:
      0   No problems occurred.
      1   Generic error code.
      2   Parse error---for instance, when parsing command-line options, the .wgetrc or .netrc...
      3   File I/O error.
      4   Network failure.
      5   SSL verification failure.
      6   Username/password authentication failure.
      7   Protocol errors.
      8   Server issued an error response.
  '''
  # Wget Options:
  # -nv                Not verbose
  # -nc                No clobber (use for OSB files to avoid 
  #                    overwriting 1-Sec with 30-Sec)
  # -np                No parent. ( Important !!! )
  # -nH                Do not create a directory named after the url
  # --cut-dirs=4       Remove the first four subdirs
  # -R "index.html*"   Do not keep the index.html* files.
  # -P xxx             xxx will be the root directory
  os.makedirs(folder, exist_ok=True)
  log_fn = datetime.datetime.now().strftime(f"{folder}/%Y-%m%d-%H%M%S-CORS-wget-{id}-log.txt")
  url_str = ' '.join(url_list)

  if clobber:
    nc_switch = ''
  else:
    nc_switch = '-nc '

  rv = sp.run( 
      'wget '\
      f'{nc_switch} '\
      f'-o {log_fn} '\
      '-nv '\
      f'-P {folder} '\
      '-np '\
      '-r '\
      '-nH '\
      '--connect-timeout=10 '\
      '--read-timeout=10 '\
      f'--cut-dirs={cut_dirs} '\
      '-R "index.html*,robots.txt,*.tmp" '\
      f'{url_str}',
        shell=True, capture_output=True)
  rv.stderr = log_fn
  rv.id = id
  return rv

In [ ]:
# Test with some sp3 and nav data.
test_url      = rinex_url
fn = f'{test_folder}/sp3_nav'
date = '2023/04/28'
print('Testing: CORS_get_Support_url_list()')
print(f'Storing in folder: {fn}\n'\
      f'             Date: {date}\n')

test_url_list = CORS_get_Support_url_list(test_url, date = date)
print('Urls downloaded')
for url in test_url_list:
  print(url)
rv = download_urls(test_url_list, folder=fn, id='sp3')
rv_str = wget_return_codes[rv.returncode]
print(f'\nOperation completed. The Wget response was: {rv_str}')

Testing: CORS_get_Support_url_list()
Storing in folder: /tmp/test//sp3_nav
             Date: 2023/04/28

Urls downloaded
https://geodesy.noaa.gov/corsdata/rinex/2023/118/IGS0OPSFIN_20231180000_01D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/IGS0OPSRAP_20231180000_01D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/IGS0OPSULT_20231180000_02D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/IGS0OPSULT_20231180600_02D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/IGS0OPSULT_20231181200_02D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/IGS0OPSULT_20231181800_02D_15M_ORB.SP3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/brdc1180.23g.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/brdc1180.23n.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/igr22595.sp3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/igs22595.sp3.gz
https://geodesy.noaa.gov/corsdata/rinex/2023/118/igu22595_00.sp3.gz
http

#| hide
### def CORS_trim_to_time(f, start, stop):

In [ ]:
#| export
def CORS_trim_to_time(
  f:str,      # RINEX file Name
  start:str,  # Start time str as `'YYYY-MM-DD HH:MM:SS.SSS'`
  stop:str):  # Stop time str  as `'YYYY-MM-DD HH:MM:SS.SSS'`
  
  """
Trims a RINEX file (f) to be between the 'start' and 'stop' times given.

This function takes in a file name (f), start time (start) and 
stop time (stop) as parameters and trims the file to the given time range.

The start and stop times are first converted to a format without colons, 
hyphens and periods. The teqc command is then used to trim the file to 
the given time range and the output is stored in a temporary file. 
The temporary file is then moved to the original file name.

Parameters:
f (str): The file name to be trimmed.
start (str): The start time in the format 'YYYY-MM-DD HH:MM:SS.SSS'.
stop (str): The stop time in the format 'YYYY-MM-DD HH:MM:SS.SSS'.

Returns:
rv (CompletedProcess): A CompletedProcess object containing information about the run command.
  """
  st = re.sub('[\:\-\.]', '', start)
  end = re.sub('[\:\-\.]', '', stop)
  print(f'CORS_trim_to_time(): f: {f}   start: {start}  stop: {stop}')
  rv = sp.run(f'teqc +out tmp.txt -st {start} -e {stop} {f}', shell=True)
  sp.run(f'mv tmp.txt {f}', shell=True)
  return rv

#| hide
### def CORS_get_station_log_url(station_list):

In [ ]:
#| export
def CORS_get_station_log_url(
  station_list  # Python `list` of CORS Stations.
) ->list:       # A list of station log file urls
  '''
    This function takes a list of station names as an argument and 
    returns a list of URLs for the station log files.

    Parameters:
    station_list (list): A list of station names.

    Returns:
    lst (list): A list of URLs for the station log files.
  '''
  lst = []
  for station in station_list:
    station = str.lower(station)
    cords_file = f'{station_log_url}{station}.log.txt'
    lst.append(cords_file)
  return lst

In [ ]:
station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
local_test_folder = test_folder
rv = CORS_get_station_log_url( station_list )
display(rv)
print('Done.')

['https://geodesy.noaa.gov/corsdata/station_log/ncdu.log.txt',
 'https://geodesy.noaa.gov/corsdata/station_log/ncbe.log.txt',
 'https://geodesy.noaa.gov/corsdata/station_log/ncrt.log.txt',
 'https://geodesy.noaa.gov/corsdata/station_log/loy2.log.txt']

Done.


#| hide
### def down_load_cors_station_log( dir, station_list):

In [ ]:
#| export
def down_load_cors_station_log( 
  dir:str,            # Existing folder to place the file.
  station_list:list   # Python `list` of CORS Stations.
) ->list:             # a `dict` where `key` is the station ID, and `value` is path of the downloade file               
  '''
    This function downloads the log files of the given CORS stations 
    from the NGS website.

    Parameters:
    dir (str): The directory where the log files should be downloaded.
    station_list (list): A list of CORS station names.

    Returns:
    rv (dict): A dictionary containing the station name as the key and 
    the path of the downloaded log file as the value.
  '''
  log_file_list = CORS_get_station_log_url(station_list)
  rv = download_urls(log_file_list, dir, id='station', cut_dirs=1)
  return rv

In [ ]:
station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
local_test_folder = test_folder
rv = down_load_cors_station_log( local_test_folder, station_list )
print(f'{wget_return_codes[rv.returncode]} See: {rv.stderr}' )
print('Done.')

0, No problems occurred. See: /tmp/test//2023-0615-123209-CORS-wget-station-log.txt
Done.


#| hide
### def CORS_get_station_plots_url(station_list):

In [ ]:
#| export
def CORS_get_station_plots_url(
  station_list): # Python `list` of CORS Stations.
  '''
    This function takes a list of station names as an argument and 
    returns a list of URLs for the corresponding station plot 
    files.

    Parameters:
    station_list (list): A list of station names.

    Returns:
    lst (list): A list of URLs for the corresponding station plot
     files.
  '''
  # Example: https://geodesy.noaa.gov/corsdata/Plots/ab02_14.short.png
  lst = []
  for station in station_list:
    station = str.lower(station)
    short_plots_file = f'{plots_url}{station}_14.short.png'
    long_plots_file  = f'{plots_url}Longterm/{station}_14.long.png'
    lst.append(short_plots_file)
    lst.append(long_plots_file)
  return lst

In [ ]:
print('Testing: CORS_get_station_plots_url()')
station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
rv = CORS_get_station_plots_url( station_list )
display(rv)
print('Done.')

Testing: CORS_get_station_plots_url()


['https://geodesy.noaa.gov/corsdata/Plots/ncdu_14.short.png',
 'https://geodesy.noaa.gov/corsdata/Plots/Longterm/ncdu_14.long.png',
 'https://geodesy.noaa.gov/corsdata/Plots/ncbe_14.short.png',
 'https://geodesy.noaa.gov/corsdata/Plots/Longterm/ncbe_14.long.png',
 'https://geodesy.noaa.gov/corsdata/Plots/ncrt_14.short.png',
 'https://geodesy.noaa.gov/corsdata/Plots/Longterm/ncrt_14.long.png',
 'https://geodesy.noaa.gov/corsdata/Plots/loy2_14.short.png',
 'https://geodesy.noaa.gov/corsdata/Plots/Longterm/loy2_14.long.png']

Done.


#| hide
### def down_load_cors_plots( dir, station_list):

In [ ]:
#| export
def down_load_cors_plots( 
  dir:str,          # Existing folder to place the file.
  station_list:list # Python `list` of CORS Stations.
):
  '''
    This function downloads the plots of the CORS stations from
     the NGS website.

    Parameters:
        dir (str): The directory where the files will be downloaded.
        station_list (list): A list of CORS station names.

    Returns:
        rv (list): A list of the downloaded files.
  '''
  plots_file_list = CORS_get_station_plots_url(station_list)
  rv = download_urls(plots_file_list, dir, id='Plots', cut_dirs=1)
  return rv

In [ ]:
print('Testing: down_load_cors_plots_coords()')
station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
local_test_folder = test_folder
rv = down_load_cors_plots( local_test_folder, station_list )
print(f'{wget_return_codes[rv.returncode]} See: {rv.stderr}' )
print('Done.')

Testing: down_load_cors_plots_coords()
8, Server issued an error response. See: /tmp/test//2023-0615-123209-CORS-wget-Plots-log.txt
Done.


#| hide
### def CORS_get_station_coords_url(dir, station_list):

In [ ]:
#| export
def CORS_get_station_coords_url(
  station_list:list,  # Python `list` of CORS Stations.
) ->list:             # List of coordinates urls
  '''
    This function takes a list of station names as an argument and 
    returns a list of URLs for the corresponding station coordinates 
    files.

    Parameters:
    station_list (list): A list of station names.

    Returns:
    lst (list): A list of URLs for the corresponding station coordinates
     files.
  '''
  lst = []
  for station in station_list:
    station = str.lower(station)
    cords_file = f'{coords_url}{station}_14.coord.txt'
    lst.append(cords_file)
  return lst

Example usage:

In [ ]:
print('Testing: CORS_get_station_coords_url()')
station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
rv = CORS_get_station_coords_url( station_list )
display(rv)
print('Done.')

Testing: CORS_get_station_coords_url()


['https://geodesy.noaa.gov/corsdata/coord/coord_14/ncdu_14.coord.txt',
 'https://geodesy.noaa.gov/corsdata/coord/coord_14/ncbe_14.coord.txt',
 'https://geodesy.noaa.gov/corsdata/coord/coord_14/ncrt_14.coord.txt',
 'https://geodesy.noaa.gov/corsdata/coord/coord_14/loy2_14.coord.txt']

Done.


#| hide
### def down_load_cors_station_coords( dir, station_list):

In [ ]:
#| export
def down_load_cors_station_coords( 
  dir:str,             # Existing folder to place the file.
  station_list:list ): # Python `list` of CORS Stations.
  '''
    This function downloads the coordinates of the CORS stations from the NGS website.

    Parameters:
        dir (str): The directory where the files will be downloaded.
        station_list (list): A list of CORS station names.

    Returns:
        rv (list): A list of the downloaded files.
  '''
  coords_file_list = CORS_get_station_coords_url(station_list)
  rv = download_urls(coords_file_list, dir, id='coords', cut_dirs=2)
  return rv

In [ ]:
print('Testing: down_load_cors_station_coords()')
station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
local_test_folder = test_folder
rv = down_load_cors_station_coords( local_test_folder, station_list )
print(f'{wget_return_codes[rv.returncode]} See: {rv.stderr}' )
print('Done.')

Testing: down_load_cors_station_coords()
0, No problems occurred. See: /tmp/test//2023-0615-123209-CORS-wget-coords-log.txt
Done.


#| hide
### def clean_up_CORS(rdir, date):

In [ ]:
#| export
def clean_up_CORS(rdir:str,  # Directory start in.
                  date:str): # Date of files to clean up.
  """
  clean_up_CORS(rdir, date)
  Inputs:
  rdir      Directory to start the find command in.
  date      The date of the CORS files to cleanup.

  Outputs:
            Removes unnecessary files.
  Returns:
            None.

  Uses the Linux 'find' command to locate and remove unnecessary files 
  from within a CORS station directory.  
  """
  Year = date.split('/')[0][2:4]
  rv = sp.check_output(f'cd {rdir}; find -name *.md5 -delete; find -name *.{Year}d -delete; find -name *.md5* -delete; find -name *.{Year}S* -delete;', shell=True) 
  return rv


#| hide
### def down_load_list_of_obs_files( folder='./', date='' station_list=[] ): 

In [ ]:
#| export
def down_load_list_of_obs_files( 
  folder='./',       # Existing folder to store downloaded CORS data.
  date='',           # Date to download. Format: 'YYYY:MM:DD'
  station_list=[] ): # List of CORS station identifiers.
  '''
    down_load_list_of_obs_files(folder, date, station_list)

    This function downloads a list of observation files from a given URL for a given date and station list.

    Parameters:
    folder (str): The folder where the files should be downloaded.
    date (str): The date for which the files should be downloaded.
    station_list (list): A list of stations for which the files should be downloaded.

    Returns:
    rv (list): A list of the downloaded files.
  '''
  # Create a string from the station list.
  station_url_list = []
  for station in station_list:
    data_url = '/'.join([ rinex_url+compute_day_of_year( date )[0], station.lower()+'/'] )
    station_url_list.append(data_url)
  
  # download the files.
  rv = download_urls(station_url_list, folder=folder,  id='OBS')
  return rv

#| hide
### def down_load_cors_data( folder = '/', date   = '', station_list = [] ):

In [ ]:
#| export
def down_load_cors_data( 
  folder:str = '/tmp',      # Existing folder to store downloaded CORS data.
  date:str = '',            # Date to download. Format: 'YYYY:MM:DD'
  progress_callback = None, # User defined function to report progress
  station_list:list = []    # List of CORS station identifiers.
  ) ->list: 
  """
  Download CORS RINEX data for the designated `date`, specified identifiers in the
  `station_list`, and store
  the data in the given `folder`.
  """
  
  def show_progress(msg):
    print(msg)
    
  # Switch to the user defined progress function if one was supplied.
  if progress_callback:
    show_progress = progress_callback

  target_folder = f'{folder}/{compute_day_of_year(date)[1]}-CORS'
  show_progress(f'saving to: {target_folder}')
  rv_lst = []

  show_progress('Downloading info on selected stations.')
  rv = download_urls(cors_info_url_list, target_folder, id='info')
  rv_lst.append(rv)

  show_progress('Downloading selected OBS files.')
  rv = down_load_list_of_obs_files( 
    folder=target_folder, 
    date=date, 
    station_list=station_list
  )
  rv_lst.append(rv)

  show_progress('Downloading sp3 & nav files.')
  url_list = CORS_get_Support_url_list( rinex_url, date = date )
  sp3_nav_folder = '/'.join([target_folder, 'sp3_nav'])
  rv = download_urls(url_list, folder = sp3_nav_folder, id='sp3')
  rv_lst.append(rv)

  show_progress('Downloading station coords, logs, and plots.')
  for func in [down_load_cors_station_coords, 
               down_load_cors_station_log,
               down_load_cors_plots ]:
    rv = func(target_folder, station_list)
    rv_lst.append(rv)

  show_progress('All downloads completed.')
  return rv_lst